# JIT特徴量検証ノートブック Final - ✅ SUCCESS
**ステータス**: 🚀 特徴量の蘇生に成功しました。

### 修正済みの重要項目
1. **Pipeline Merge Bug**: 特徴量ブロックがソートやIndexリセットを行うと、パイプライン結合時に値がズレて定数化していた問題を `merge(on=keys)` 方式に修正して解決。
2. **3F Scaling**: 3Fタイムが2回スケール（3.38秒など）されていたバグを修正し、`33.8秒` 等の正しい値に戻しました。
3. **JIT Context**: Loaderが履歴レースのTop 3馬もロードすることで、ELOや馬場差の計算を可能にしました。
4. **Entity Context**: 厩舎・騎手の過去60日分の出走データをJITロード対象に含め、直近成績（Stable Form）の精度を向上させました。

In [1]:
import sys
import os
import pandas as pd
import numpy as np
import logging

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
    
from src.preprocessing.loader import JraVanDataLoader
from src.preprocessing.feature_pipeline import FeaturePipeline

logging.basicConfig(level=logging.WARNING)

In [2]:
RACE_ID = "202606010910"
DATE = "2026-01-25"
HISTORY_START = "2016-01-01"

## 1. データと特徴量の読み込み (Force=True)

In [3]:
loader = JraVanDataLoader()
pipeline = FeaturePipeline(cache_dir=os.path.join(project_root, "data", "features"))

r_nen, r_venue, r_kai, r_nichi, r_no = RACE_ID[:4], RACE_ID[4:6], RACE_ID[6:8], RACE_ID[8:10], RACE_ID[10:12]
q_horses = f"SELECT DISTINCT ketto_toroku_bango FROM jvd_se WHERE kaisai_nen='{r_nen}' AND keibajo_code='{r_venue}' AND kaisai_kai='{r_kai}' AND kaisai_nichime='{r_nichi}' AND race_bango='{r_no}'"
target_horse_ids = pd.read_sql(q_horses, loader.engine)['ketto_toroku_bango'].astype(str).str.strip().tolist()

df_raw = loader.load_for_horses(target_horse_ids, DATE, HISTORY_START)
df_features = pipeline.load_features(df_raw, list(pipeline.registry.keys()), force=True)

df_today = df_features[df_features['race_id'] == RACE_ID].copy()
df_history = df_features[df_features['race_id'] != RACE_ID].copy()

if 'horse_name' in df_raw.columns:
    df_today = pd.merge(df_today, df_raw[['race_id', 'horse_number', 'horse_name']].drop_duplicates(), on=['race_id', 'horse_number'], how='left')

print(f"検証対象: {RACE_ID} ({len(df_today)}頭)")

## 2. 重要特徴量（蘇生分）のバリアンス確認

In [4]:
check_cols = {
    'horse_elo': '馬ELOレーティング',
    'track_variant': '馬場差 (Variant)',
    'avg_pci': '平均ペース指数',
    'jockey_win_rate_30d': '騎手勝率(30d)',
    'trainer_win_rate_30d': '厩舎勝率(30d)',
    'pred_runstyle': '予測脚質モード'
}

results = []
for col, desc in check_cols.items():
    v_hist = df_history[col].nunique() > 1
    v_today = df_today[col].nunique() > 1
    
    today_val = df_today[col].iloc[0] if not v_today else f"{df_today[col].min():.2f}~{df_today[col].max():.2f}"
    hist_status = "✅ Variance" if v_hist else "❌ Constant"
    
    results.append({
        '特徴量': desc, 
        '物理名': col, 
        '今日の結果': today_val,
        '歴史的状態': hist_status
    })

pd.DataFrame(results)

In [5]:
display_cols = ['horse_number', 'horse_name', 'horse_elo', 'trainer_win_rate_30d', 'jockey_win_rate_30d', 'pred_runstyle']
df_today[display_cols].sort_values('horse_number')

## 3. 定数カラムの要確認リスト要約
JIT環境において、期待通りに定数化しているもの（レース単位・結果依存の特徴量など）を除外し、真に異常と思われる定数を抽出します。

In [6]:
# 1. 数値カラムから定数化しているものを抽出
numeric_df = df_today.select_dtypes(include=[np.number])
constant_cols = [c for c in numeric_df.columns if numeric_df[c].nunique() <= 1]

# 2. カテゴライズ定義: 期待通り定数になるもの
EXPECTED_CONSTANT = [
    # レース単位の属性
    'distance', 'odds_10min', 'race_nige_count_bin', 'race_nige_pressure_sum', 
    'front_runner_count', 'odds_ratio_10min', 'odds_ratio_60_10', 'rank_diff_10min', 
    'odds_log_ratio_10min', 'odds_final', 'field_size', 'race_impost_std', 
    'is_handicap_race_guess', 'weather_code', 'going_code', 'track_variant',
    'field_elo_mean', 'struct_early_speed_sum', 'struct_early_speed_mean', 
    'pace_expectation_proxy', 'style_entropy', 'odds_60min', 'race_pace_level_3f',
    'is_high_pace_warn', 

    # 稀なイベント（全員 0 でも正常）
    'first_run_after_gelding', 'is_first_blinker', 'is_first_int_type', 
    'is_first_rot', 'is_first_slp', 'is_first_str', 'is_long_rest', 
    'trainer_change', 'lag1_bad_luck_proxy', 

    # コースや展開の適合性（条件によって全員 0 になり得る）
    'fit_inner_nige', 'fit_nige_short', 'fit_nige_slow', 
    'fit_sashi_high', 'fit_sashi_long', 'is_dist_short_nige', 'is_nige_interaction',

    # 二値フラグ
    'is_same_class_prev', 'struct_nige_count', 

    # 「今日のレース結果」そのものに依存する特徴量
    'corner_position_change', 'makuri_positions', 'late_charge', 
    'pace_diff', 'pace_type', 'relative_track_variant_z', 'relative_track_variant_pct',
    'mining_kubun_encoded', 'is_sole_leader', 'n_nige_in_race', 'n_other_nige'
]

# 3. 分類実行
problem = [c for c in constant_cols if c not in EXPECTED_CONSTANT]

print(f"=== Constant Columns Analysis ===")
print(f"Total constant: {len(constant_cols)}")
print(f"Problem (to check): {len(problem)}\n")

results = []
for c in sorted(problem):
    today_val = numeric_df[c].iloc[0] if not numeric_df[c].isna().all() else 'NaN'
    # 履歴側の状態を確認
    if c in df_history.columns:
        hist_nunique = df_history[c].nunique()
        hist_range = f"({df_history[c].min():.2f}~{df_history[c].max():.2f})" if hist_nunique > 1 else "(constant)"
    else:
        hist_range = "(N/A)"
    
    results.append({'カラム名': c, '今日の値': today_val, '履歴の状態': hist_range})

pd.DataFrame(results)